In [1]:
2 + 2

4

In [2]:
import os
import pyarrow as pa
import pandas as pd

In [3]:
import joblib
import xgboost as xgb

In [4]:
# Define the directory path
data_dir = os.path.join('C:\\', 'Users', 'KonuTech', 'zoomcamp-capstone-01', 'data')
data_dir

'C:\\Users\\KonuTech\\zoomcamp-capstone-01\\data'

In [5]:
# Step 4: Convert and save the combined DataFrame as a single Parquet file
combined_parquet_file = 'test_data.parquet'

In [6]:
# Step 6: EDA on the combined Parquet file
parquet_df = pd.read_parquet(os.path.join(data_dir, combined_parquet_file))  # Read the combined Parquet file

In [7]:
# Shuffle the entire DataFrame randomly
parquet_df = parquet_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
unique_clients_df = parquet_df.drop_duplicates(subset='customer_ID', keep='first')

In [9]:
unique_clients_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 924621 entries, 0 to 11363265
Data columns (total 190 columns):
 #    Column       Dtype  
---   ------       -----  
 0    customer_ID  object 
 1    S_2          object 
 2    P_2          float64
 3    D_39         float64
 4    B_1          float64
 5    B_2          float64
 6    R_1          float64
 7    S_3          float64
 8    D_41         float64
 9    B_3          float64
 10   D_42         float64
 11   D_43         float64
 12   D_44         float64
 13   B_4          float64
 14   D_45         float64
 15   B_5          float64
 16   R_2          float64
 17   D_46         float64
 18   D_47         float64
 19   D_48         float64
 20   D_49         float64
 21   B_6          float64
 22   B_7          float64
 23   B_8          float64
 24   D_50         float64
 25   D_51         float64
 26   B_9          float64
 27   R_3          float64
 28   D_52         float64
 29   P_3          float64
 30   B_10         float64


In [10]:
unique_clients_df.to_parquet(os.path.join(data_dir, "test_data_randomly_sampled.parquet"))

In [11]:
!ls

01_prepare_train_data.ipynb
02_eda.ipynb
03_downsample_data.ipynb
04_get_champion_binary_classifier.ipynb
05_prepare_test_data.ipynb
06_score_test_data.ipynb
grid_search_results_2023-11-01_17-54-58.json
grid_search_results_2023-11-01_20-08-34.json
LogisticRegression_2023-11-01_17-54-58.bin
LogisticRegression_2023-11-01_20-08-34.bin
RandomForest_best_model.bin
training_log.log
XGBoost_2023-11-01_17-54-58.bin
XGBoost_2023-11-01_20-08-34.bin


In [12]:
model_file = "XGBoost_2023-11-01_20-08-34.bin"

In [13]:
# columns_used = ['B_16', 'B_20', 'B_3', 'B_38', 'B_7', 'B_9', 'D_44', 'D_48', 'D_55', 'D_75']
columns_used = ['B_16', 'B_19', 'B_20', 'B_23', 'B_3', 'B_38', 'B_4', 'B_7', 'B_9', 'D_44', 'D_48', 'D_55', 'D_58', 'D_74', 'D_75']
columns_used

['B_16',
 'B_19',
 'B_20',
 'B_23',
 'B_3',
 'B_38',
 'B_4',
 'B_7',
 'B_9',
 'D_44',
 'D_48',
 'D_55',
 'D_58',
 'D_74',
 'D_75']

In [14]:
def score_xgb_model(model_file, input_data, columns_used):
    # Load the XGBoost model from the specified file
    xgb_model = joblib.load(model_file)
    
    # Ensure the input DataFrame contains only the specified columns
    input_data = input_data[columns_used]
    
    # Make predictions using the loaded model
    # predictions = xgb_model.predict(input_data)
    predictions = xgb_model.predict_proba(input_data)  # Use predict_proba to get probabilities

    # Extract the probability of the positive class (assuming binary classification)
    probabilities = predictions[:, 1]
    
    # Add predictions to the input data DataFrame
    # input_data['prediction'] = predictions

    # Add probabilities to the input data DataFrame
    input_data['probability'] = probabilities
    
    return input_data

In [15]:
df_scored = score_xgb_model(
    model_file=model_file,
    input_data=unique_clients_df,
    columns_used=columns_used
)

c:\users\konutech\zoomcamp-capstone-01\venv\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


ValueError: Expected a 1D array, got an array with shape (924621, 2)

In [ ]:
df_scored.info(verbose=True)

In [ ]:
df_scored

In [ ]:
# Left join df_scored to unique_clients_df on row index
merged_df = unique_clients_df["customer_ID"].to_frame().join(df_scored["prediction"].to_frame(), how='left')

In [ ]:
merged_df

In [ ]:
merged_df.to_csv("submission_2023-11-01_20-08-34.csv", index=False)